# Project - Super Mario Bros. Game (Part 1)

In this project, you'll make your own version of a Super Mario Bros. game. For the first part, we'll focus on the mechanics of the game — setting up the scene of one level, moving your character around, and keeping track of lives. We'll also get to add some simple power-ups and interaction with an enemy.

[![Mario Game Part 1](Images/final-p1.png)](Images/final-p1.mov)

---

# Learning Objectives

* Students will apply knowledge of 2D physics (colliders, rigidbody, etc.) to a game project.
* Students will apply object-oriented programming concepts including data types, variables, conditionals, and inputs.
* Students will create a basic game to be built upon in part 2.
---

# Sections

Section 1
* Set-up
* Character and Scene
* Player Movement
* Platform Motion

Section 2
* Update Player Movement
* Power-ups
* Enemies
* Lives

Section 3
* Menu
* In-Game Interface
* Level 2
* Build Your Game!

---

# Section 1

## Set-up

1. Open Unity and create a 2D game.
2. Download assets. You can use the ones provided or find your own. You'll want to have a variety of scene options (like bricks, grass, etc.), at least one player character sprite, and at least one enemy sprite.
3. Make sure these assets are in the Assets folder of your project.

![Unity Hub Setup Screen](Images/unity-setup.png)

## Character and Scene

1. Add Mario to the scene.
1. Add your environment elements.
1. Edit the sizes of your elements to be proportional with each other and the camera frame. Build until you're happy with how the level looks. See screenshots for an example.
1. Change the background color.
1. Add colliders on all of your elements.
1. Configure your elements so that Mario can be affected by gravity but the environment pieces are static. Also, Mario should never rotate upside down, so set that constraint as well.

**Solution**

Add assets by dragging in the sprites. To get pieces to line up properly, click on the elements and adjust their Transform Position to match in the desired dimension (x for lining up vertically, y for horizontally, z for nearness).

Background color can be changed by clicking on Main Camera and changing the Background. (see image for where that is in the Inspector)

When finished, all elements should have some type of Collider 2D component and a Rigidbody 2D component. Mine all use Box Collider 2D for simplicity. On environment elements, the Rigidbody 2D should be set to Static. On Mario, it should be Dynamic. Also, under constraints, Mario should have Fixed Rotation, so the box for Z should be checked.

![alt text](Images/background.png)

## Platform Motion

Let's add some motion to a couple of the platforms. We're going to have the highest platform move horizontally and the one above the gap move vertically.

1. Make sure that the bricks that make up each platform are grouped together. This will allow you to put a script only on the parent brick and move the entire platform together.
1. Add a script that moves the desired platforms right to left or up and down over time. You'll need to have a variable keeping track of how far the platform has moved so that you know when to have it go the other direction.
1. Add the script (provided below) to the script that controls Mario's movement. You'll need this to make Mario's movement follow the moving platforms so he sticks on the platform instead of the platform moving out from underneath him. Change the names in the if statement to match the names of the parent bricks of your moving platforms.

In [ ]:
void OnCollisionEnter2D(Collision2D other) {
    if (other.gameObject.name == "brick (3)" || other.gameObject.name == "brick (4)") {
        transform.parent = other.transform;
    }
}

void OnCollisionExit2D(Collision2D other) {
    transform.parent = null;
}

**Solution**

Your scripts should look something like this:

In [ ]:
# horizontal movement
using System.Collections;
using System.Collections.Generic;
using UnityEngine;

public class move_horizontal : MonoBehaviour
{
    public float amount = 2.0f;
    public float speed = 1.0f;

    Vector3 direction = Vector3.left;
    float moved = 0.0f;

    // Update is called once per frame
    void Update()
    {
        if (moved > amount || moved < -amount) {
          direction = -direction;
          moved = 0.0f;
        }
        transform.position += direction * speed * Time.deltaTime;
        moved += speed * Time.deltaTime;
    }
}

In [ ]:
# vertical movement
using System.Collections;
using System.Collections.Generic;
using UnityEngine;

public class move_vertical : MonoBehaviour
{
  public float amount = 3.0f;
  public float speed = 1.0f;

  Vector3 direction = Vector3.up;
  float moved = 0.0f;

  // Update is called once per frame
  void Update()
  {
      if (moved > amount || moved < -amount) {
        direction = -direction;
        moved = 0.0f;
      }
      transform.position += direction * speed * Time.deltaTime;
      moved += speed * Time.deltaTime;
  }
}

## Player Movement

1. Create a script that will allow Mario to move right and left, and jump.
1. Make the speed and the jump height public variables so that you can adjust them in the Inspector.

**Solution**

<a href="Images/mario-movement.mov"><img src="https://nutritionbymia.com/wp-content/uploads/play-button-icon-png-0.png" width=150px></img></a>

Your code should look something like this:

In [ ]:
using System.Collections;
using System.Collections.Generic;
using UnityEngine;

public class mario_movement : MonoBehaviour
{
    public float speed = 0.1f;
    public float jump_height = 0.1f;

    void FixedUpdate()
    {
        if (Input.GetKey("right")) {
          transform.position += Vector3.right * speed;
        }

        if (Input.GetKey("left")) {
          transform.position += Vector3.left * speed;
        }

        if (Input.GetKey("up")) {
          transform.position += Vector3.up * jump_height;
        }
    }
}

# Section 2

## Update Player Movement

Now that you've learned about using AddForce() instead of transform, let's modify Mario's movement.

1. Change Mario to use AddForce() on the RigidBody instead of transform when jumping.
1. Limit the number of times Mario can jump. You learned in your lesson how to check if the character is currently touching the ground — for this game, we'll want to allow double jumping, so add a private variable that keeps track of how many jumps and resets upon hitting something else. We don't have to worry about what Mario has hit because the jump in the Mario games resets upon hitting anything, though feel free to change this mechanism if you want.
    * Note: In order to prevent the number of jumps from counting multiple times, use Input.GetKeyDown instead of Input.GetKey. Also, this needs to happen in Update() instead of FixedUpdate() to prevent the Input from triggering multiple times for one click.
1. Also, change to using forces instead of transform for horizontal movement as well.

**Solution**

Your code should look something like this:

In [ ]:
using System.Collections;
using System.Collections.Generic;
using UnityEngine;

public class mario_movement : MonoBehaviour
{
    public float speed = 120.0f;
    public float jumpForce = 5.0f;

    private Rigidbody2D rb;
    [SerializeField] private int jumps = 0;

    void Start() {
        rb = GetComponent<Rigidbody2D>();
    }

    void FixedUpdate() {
        Walk();
    }

    void Update() {
        // Jump if you're pressing Up and the number of jumps is less than 2
        if (Input.GetKeyDown(KeyCode.UpArrow) && jumps < 2) {
            Jump();
        }
    }

    private void Walk() {
        float horizontal = Input.GetAxisRaw("Horizontal");
        Vector2 movement = new Vector2(horizontal * speed * Time.deltaTime, rb.velocity.y);
        rb.velocity = movement;
    }

    private void Jump() {
        rb.velocity = new Vector2(rb.velocity.x, 0);
        rb.AddForce(jumpForce * Vector2.up * rb.mass, ForceMode2D.Impulse);
        jumps++;
    }

    void OnCollisionEnter2D(Collision2D other) {
        jumps = 0;

        if (other.gameObject.name == "brick (3)" || other.gameObject.name == "brick (4)") {
            transform.parent = other.transform;
        }
    }

    void OnCollisionExit2D(Collision2D other) {
        transform.parent = null;
    }
}


## Power-ups

1. Add a grow power-up sprite to the scene. Add components to make it affected by gravity and able to collide with other objects. Constrain it so it doesn't rotate, and hide it for now.
1. Create a script that will make the power-up show up after Mario hits the question mark block.
    * Hint: The script should go on Mario to check if he has collided with the question mark block.
1. Create a script that will check if Mario has collided with the power-up and increase his size when he does. Add a public boolean variable that will tell if he has this power-up activated (we'll need that later to decide what happens when he runs into enemies). Also, make the power-up disappear after Mario has run into it.

1. Do the same for the shrink power-up (but make him shrink instead of grow!). You should not need to make any more scripts on Mario — instead, modify the scripts you already have to account for the new power-up.

**Solution**

<a href="Images/power-ups.mov"><img src="https://nutritionbymia.com/wp-content/uploads/play-button-icon-png-0.png" width=150px></img></a>

The power-ups should have a Collider 2D and Rigidbody 2D, with the Z constraint checked. They should start out inactive, which can be done by unchecking the box next to the name. That will make the sprite disappear from the scene and be grayed out in the Hierarchy panel.

![Uncheck the box next to the power up sprite name in the Inspector panel](Images/power-ups.png)

The scripts can all go on Mario, since the collisions both are with the player character and affect the Mario's size. I have them all combined into one script, but they can be separated out. It should look something like this:

In [ ]:
using System.Collections;
using System.Collections.Generic;
using UnityEngine;

public class activate_powerups : MonoBehaviour
{
    public GameObject question_block;

    public GameObject grow_powerup;
    public bool grow_active;

    public GameObject shrink_powerup;
    public bool shrink_active;

    Vector3 original_scale; // this is used so that the power-ups override each other instead of cancelling out
 
    // Start is called before the first frame update
    void Start()
    {
      original_scale = transform.localScale;
    }

    void OnCollisionEnter2D(Collision2D other) {
        if (other.gameObject == question_block && transform.position.y < question_block.transform.position.y) {
            // the second condition checks that Mario is jumping up to hit the question block
            grow_powerup.SetActive(true);
            shrink_powerup.SetActive(true);
        }

        if (other.gameObject == grow_powerup) {
          grow_active = true;
          shrink_active = false;
          transform.localScale = original_scale + new Vector3(0.1f, 0.1f, 0.1f);
          grow_powerup.SetActive(false);
        }

        if (other.gameObject == shrink_powerup) {
          shrink_active = true;
          grow_active = false;
          transform.localScale = original_scale + new Vector3(-0.1f, -0.1f, -0.1f);
          shrink_powerup.SetActive(false);
        }
    }
}

*Note that in order to make this work, the public GameObjects need to be connected properly. This can be done in the Inspector of the Unity Editor (see image).

![Connect GameObjects to the public GameObjects used in scripts by dragging from Hierarchy to Inspector](Images/connect-GameObjects.png)

## Enemies

1. Add an enemy goomba to the scene. Place two additional bricks around it so that the goomba is limited to a confined space.
1. Scale its size and make sure the goomba and the bricks can collide with other objects and be affected by gravity.
1. Create a script that will make your goomba walk back and forth on the path between the two bricks.
  * Hint: You can check for collisions with the bricks to see if the goomba should change direction.
1. Create a script on Mario that will check if Mario has collided with the goomba. Call it something like "mario_lives" since we'll also be using this to track Mario's lives. For now, have the program print a debug statement that says "Collided with goomba!" when Mario hits the goomba.

**Solution**

The enemy goomba and the bricks should have Collider 2D and RigidBody 2D components. The bricks should be static, but the goomba should be dynamic since it will be moving and colliding with static objects.

The code for walking back and forth should look something like this:

In [ ]:
using System.Collections;
using System.Collections.Generic;
using UnityEngine;

public class goomba_walking : MonoBehaviour
{
    public float speed = 0.5f;
    public GameObject right_brick;
    public GameObject left_brick;
    Vector3 direction = Vector3.right;

    // Update is called once per frame
    void Update()
    {
        transform.position += direction * speed * Time.deltaTime;
    }

    void OnCollisionEnter2D(Collision2D other) {
        if (other.gameObject == right_brick) {
          direction = Vector3.left;
        }
        else if (other.gameObject == left_brick) {
          direction = Vector3.right;
        }
    }
}


The script for Mario and goomba collisions should look something like this. Again make sure that the public GameObjects are connected properly on both of these scripts.

In [ ]:
using System.Collections;
using System.Collections.Generic;
using UnityEngine;

public class mario_lives : MonoBehaviour
{
    public GameObject goomba;
    public int lives = 5;

    void OnCollisionEnter2D(Collision2D other) {
        if (other.gameObject == goomba) {
            Debug.Log("Collided with goomba!");
        }
    }
}


## Lives

1. Add a public variable "lives" to the mario_lives script. Set it equal to 5 to start.
1. Replace the Debug.Log in the mario_lives script with code that will decrease the number of lives by 1 and kill (deactivate) Mario UNLESS Mario jumps on top of the goomba. If Mario lands on top of the goomba, the goomba should die (by deactivating).
  * Hint: To check if Mario is on top of the goomba, check if the lowest point on Mario is above or at the same level as the highest point of the goomba using the collider's bounds.

Mario can also lose lives in other ways. For example, if he falls through the hole in the middle of the scene. We can check for that by looking at the y-value of Mario's transform.position.

3. Create an empty GameObject called worldScripts. We'll need this to control what happens when Mario (or another character) dies.
  * Since this GameObject is empty, it won't show up or otherwise interfere with your game. However, it is necessary to reactivate Mario once he's been deactivated. It's also very useful for connecting different objects in your scene and later passing information between levels.
1. Create a script that checks if Mario's y position is low enough that he is off screen. If he is, Mario should lose a life and be deactivated. Any power-ups should also be canceled out.
1. This same script should also check if Mario is deactivated. It should move Mario back to the initial starting position and reactivate him if he still has lives left. If Mario has no more lives, print a message to the console stating, "Game Over. You have no more lives left." and stop the game.
  * Hint: Use [Time.timeScale](https://docs.unity3d.com/ScriptReference/Time-timeScale.html) to stop the game. To get the desired behavior, also make sure that your goomba walking script has a Time.deltaTime multiplied into the speed and the script on the worldScripts object uses FixedUpdate() instead of Update().

**Solution**

<a href="Images/deaths.mov"><img src="https://nutritionbymia.com/wp-content/uploads/play-button-icon-png-0.png" width=150px></img></a>

Note that while not shown in the video, the goomba should stop moving when the game stops — that should be covered by multiplying by Time.deltaTime.

The mario_lives script should end up looking something like this. I've used the [collider's bounds](https://docs.unity3d.com/2019.2/Documentation/ScriptReference/Bounds.html) to figure out what the top and bottom most values for the goomba and Mario, respectively, and compared those to figure out if the goomba should die or if Mario should lose a life.

In [ ]:
using System.Collections;
using System.Collections.Generic;
using UnityEngine;

public class mario_lives : MonoBehaviour
{
    public GameObject goomba;
    public int lives = 5;

    void OnCollisionEnter2D(Collision2D other) {
        if (other.gameObject == goomba) {
            if (GetComponent<BoxCollider2D>().bounds.min.y >= goomba.GetComponent<BoxCollider2D>().bounds.max.y) {
                goomba.SetActive(false);
            }
            else {
              lives -= 1;
              gameObject.SetActive(false);
            }
        }
    }
}


The script on the worldScripts object should look something like this:

In [ ]:
using System.Collections;
using System.Collections.Generic;
using UnityEngine;

public class mario_deaths : MonoBehaviour
{
    public GameObject mario;
    Vector3 mario_start;
    Vector3 mario_size;

    // Start is called before the first frame update
    void Start()
    {
        mario_start = mario.transform.position;
        mario_size = mario.transform.localScale;
    }

    void FixedUpdate()
    {
        if (mario.transform.position.y < -6) {
          mario.GetComponent<mario_lives>().lives -= 1;
          mario.SetActive(false);
        }
        if (!mario.activeInHierarchy && mario.GetComponent<mario_lives>().lives > 0) {
          mario.transform.position = mario_start;
          mario.transform.localScale = mario_size;
          mario.GetComponent<activate_powerups>().grow_active = false;
          mario.GetComponent<activate_powerups>().shrink_active = false;
          mario.SetActive(true);
        }
        else if (mario.GetComponent<mario_lives>().lives <= 0) {
          Debug.Log("Game Over. You have no more lives left.");
          Time.timeScale = 0;
        }
    }
}


# Section 3

## Menu

1. Make a new scene for the menu.
1. Add a Canvas with a background image.
1. Add the game logo.
1. Add a button that says "Start".
1. Create a new empty GameObject. Add a script that will go to the first level when you click start.

**Solution**
1. Go to File > New Scene and save the scene as.
1. Go to GameObject > UI > Canvas to create the canvas object (or directly go to  GameObject > UI > Image which should also automatically create the canvas).
    1. Optional: rename the background image to background.
    1. Drag in the background source image into the Source Image slot in the image's Inspector. Click Set Native Size.
    1. Anchor the background to the top left and move it accordingly.
    1. Add an Aspect Ratio Fitter component and select Envelope Parent.
1. Create another image and add in the game logo here. Also set native size.
1. Create a Button element. Change its child Text element to say Start. Edit the sizing so that the button shows up at a reasonable size.
1. Click the + under On Click () on the Button (Script) component. Drag in your scripts object and select the UIManager.StartGame function. Your UIManager script should look something like this:

In [ ]:
using System.Collections;
using System.Collections.Generic;
using UnityEngine;
using UnityEngine.SceneManagement;

public class UIManager : MonoBehaviour
{
    public void StartGame()
    {
        SceneManager.LoadScene("level-1");
    }
}

## In-Game Interface

1. Add a Canvas to Level 1. Add text components that say "Level 1" and "Lives". Style these as you wish.
1. Add a script to each of the text objects that changes the text based on what's happening in the game. To modify the text, set GetComponent<UnityEngine.UI.Text>().text to the new value.
    1. Specifically, the Level 1 text should have a script that checks what the scene name is and change the text accordingly (it'll be helpful if you name your scenes level-1, 2, etc.! If you do this, you can use String.Split("-"), and then the resulting array's second item - index 1 - should be the level number). You'll need to use the SceneManagement module for this - the SceneManager.GetActiveScene().name will tell you the name of the current scene.
    1. The Lives text should check for Mario's lives, which should be a public variable, and list that number after (i.e. Lives: 5).

**Solution**

1. Create the UI objects.
1. Your two scripts should look like this:

In [ ]:
# Level
using System.Collections;
using System.Collections.Generic;
using UnityEngine;
using UnityEngine.SceneManagement;

public class level_text : MonoBehaviour
{
    private string[] scene;

    // Update is called once per frame
    void Update()
    {
        scene = SceneManager.GetActiveScene().name.Split('-');
        GetComponent<UnityEngine.UI.Text>().text = "Level " + scene[1];
    }
}

In [ ]:
# Lives
using System.Collections;
using System.Collections.Generic;
using UnityEngine;

public class lives_text : MonoBehaviour
{
    private GameObject mario;
    // Start is called before the first frame update
    void Start()
    {
        mario = GameObject.Find("mario");
    }

    // Update is called once per frame
    void Update()
    {
        GetComponent<UnityEngine.UI.Text>().text = "Lives: " + mario.GetComponent<mario_lives>().lives;
    }
}

## Level 2

1. Duplicate level 1. Rename it to level-2. Make a modification to make it distinct from level 1 so you know if your scene switch worked.
1. Add a completion flag to level 1. Add a Collider2D to it.
1. Write a script to handle the scene switch when Mario collides with the flag.

**Solution**

You can easily duplicate the level by saving as under a different name.

Your script should look something like this:

In [ ]:
using System.Collections;
using System.Collections.Generic;
using UnityEngine;
using UnityEngine.SceneManagement;

public class level_complete : MonoBehaviour
{
    public GameObject flag;
    private string[] current;

    void OnCollisionEnter2D(Collision2D other) {
      if (other.gameObject == flag) {
        # I wrote it like this so that it would be easier for adding future levels.
        current = SceneManager.GetActiveScene().name.Split('-');
        SceneManager.LoadScene(current[0] + '-' + (int.Parse(current[1])+1));
        
        # However, this will suffice for now:
        SceneManager.LoadScene("level-2");
      }
    }
}

## Build Your Game!

Build your game for your device. Run your game!

**Solution**

1. Open Build Settings. Make sure to add all of your scenes. You may have to reopen them if they are not already added.
2. Select the desired platform (PC, Mac). Hit Build or Build and Run.

# Part 1 Complete

Great job! You now have a basic platformer-style game that you can continue to build on.